# RNN Training
This notebook can be used to train an LSTM for text classification and generate predictions for the kaggle competition found [here](https://www.kaggle.com/c/quora-insincere-questions-classification). 

The notebook utilizes Keras and GloVe for preprocessing using word embeddings. Then, Keras with Tensorflow backend is used for training a deep LSTM. 

Ensure that the train.csv and test.csv are in the data/ directory of this project. 

In [3]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook as tqdm
from time import sleep

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM
from keras.models import Sequential

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Read in training and testing data
train_df = pd.read_csv('data/train.csv')
train_df.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [9]:
# Eliminate any potential null values
train_df[train_df.isnull().any(axis=1)].shape

(0, 3)

In [10]:
# Extract the training data and corresponding labels
text = train_df['question_text'].values
labels = train_df['target'].values

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(text, labels,\
                                                  test_size=0.2)

In [11]:
embed_size = 300 # how big is each word vector
max_words = 30000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use

In [12]:
## Tokenize the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)

## Pad the sentences 
X_train = pad_sequences(X_train, maxlen=maxlen)
X_val = pad_sequences(X_val, maxlen=maxlen)

In [13]:
word_index = tokenizer.word_index
print('The word index consists of {} unique tokens.'.format(len(word_index)))

The word index consists of 196065 unique tokens.


In [4]:
# Create the embedding dictionary from the word embedding file
embedding_dict = {}
filename = os.path.join('./embeddings/', 'glove.840B.300d/glove.840B.300d.txt')
with tqdm(os.path.getsize(filename), total=os.path.getsize(filename)) as pbar:
    with open(filename) as f:
        for line in f:
            pbar.update(len(line))
            line = line.split()
            sleep(0.001)
            token = line[0]
            try:
                coefs = np.asarray(line[1:], dtype='float32')
                embedding_dict[token] = coefs
            except:
                pass
print('The embedding dictionary has {} items'.format(len(embedding_dict)))

KeyboardInterrupt: 

In [47]:
embed_mat = np.zeros(shape=[len(word_index)+1, embed_size])
for word, idx in word_index.items():
    vector = embedding_dict.get(word)
    if vector is not None:
        embed_mat[idx] = vector

In [48]:
print(embed_mat.shape)


(196467, 300)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    3   17  109
    5  384  187  168  131 5450  302  455    5   25 3236   13  253  380
  109 1077]


In [49]:
embed_layer = Embedding(len(word_index) + 1,
                        embed_size,
                        weights=[embed_mat],
                        input_length=maxlen,
                        trainable=False)

In [50]:
def create_rnn():
    model_rnn = Sequential()
    model_rnn.add(embed_layer)
    model_rnn.add(GRU(500, dropout=0.2, recurrent_dropout=0.2))
    model_rnn.add(Dense(1, activation='sigmoid'))
    model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_rnn

In [51]:
lstm = create_rnn()
lstm.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1, batch_size=128)

Train on 1044897 samples, validate on 261225 samples
Epoch 1/1


    128/1044897 [..............................] - ETA: 16:36:22 - loss: 0.7717 - acc: 0.2266

    256/1044897 [..............................] - ETA: 11:10:07 - loss: 0.6227 - acc: 0.5742

    384/1044897 [..............................] - ETA: 9:19:57 - loss: 0.5176 - acc: 0.7005 

    512/1044897 [..............................] - ETA: 8:27:10 - loss: 0.4572 - acc: 0.7598

...

  10752/1044897 [..............................] - ETA: 5:39:08 - loss: 0.2065 - acc: 0.9296

  10880/1044897 [..............................] - ETA: 5:39:05 - loss: 0.2061 - acc: 0.9297

  11008/1044897 [..............................] - ETA: 5:39:07 - loss: 0.2052 - acc: 0.9299

  11136/1044897 [..............................] - ETA: 5:39:26 - loss: 0.2045 - acc: 0.9301

KeyboardInterrupt: 

In [29]:
print(sum(y for y in y_val if y==1))
print(934*2)

113
1868


# Predictions
The remainder of this notebok will generate predictions from the test set and write them to a submission csv file. 

In [ ]:
test_df = pd.read_csv('data/test.csv')
X_test = train_df['question_text'].values

X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=maxlen)

test.to_csv('data/bmmidei_NB_Submission_1', index=False